In [1]:
%config Completer.use_jedi = False

In [1]:
%%time
import sklearn
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.svm import SVC
import pprint
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import ParameterGrid

CPU times: user 1.31 s, sys: 351 ms, total: 1.66 s
Wall time: 1.28 s


In [3]:
#df = pd.read_csv('df_for_RF.csv')
#df1 = pd.read_csv('df_1_outlier.csv')
#df = pd.read_csv('finaldf_orderd_withzeros.csv')
#df = pd.read_csv('df_eng.csv')
df = pd.read_csv('PCA_results.csv')
df.drop('Unnamed: 0', axis = 1, inplace=True)
df.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,pc11,pc12,pc13,pc14,pc15,diagadhd
0,-6.316805,-3.993883,2.608049,-1.799177,0.421655,-0.185647,-0.866421,2.557691,0.684623,1.389368,0.938905,-1.107879,-3.008717,-2.574542,2.074565,0
1,-3.483505,3.740260,3.714517,-1.483981,-0.932160,-1.223809,-0.561965,-2.268734,0.385098,-1.309618,0.762300,1.392407,-1.410966,0.480299,3.181608,0
2,-5.072733,-1.089972,-2.044674,-0.989411,4.432155,3.538149,0.327935,-1.656317,1.972287,2.179859,-3.130990,1.615621,0.998560,1.519352,-1.185252,0
3,9.102420,-0.533239,0.310449,6.574843,1.080824,-0.322345,0.917680,1.597076,0.692811,-2.357757,0.643224,3.000754,0.006578,-0.434549,1.308862,0
4,0.069206,-1.513340,0.906819,0.405307,-0.203222,2.936823,-0.677617,0.827757,1.834682,-0.723978,0.083857,-3.246837,-1.006093,0.247815,-0.204260,1


In [4]:
sizes = df['diagadhd'].value_counts(sort = 1)
sizes

0    71
1    46
Name: diagadhd, dtype: int64

In [5]:
df.shape

(117, 16)

In [6]:
X = df.iloc[:,:14].values
y = df.iloc[:,15].values

### define dependent and independent variables(adverserial validation)

In [ ]:
# train-test split evaluation of xgboost model
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# split data into X and y
#X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,2:], data['diagadhd'], test_size=0.3)
training_set, test_set = train_test_split(df, test_size = 0.2, random_state = 3)
X_train = training_set.iloc[:,3:].values
y_train = training_set.iloc[:,1].values
X_test = test_set.iloc[:,3:].values
y_test = test_set.iloc[:,1].values

# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

#### grid search for hyperparameter tuning

In [7]:
from sklearn.model_selection import RandomizedSearchCV

# number of trees in RF
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 91)]

# number of features to consider at every split
max_features = ['auto', 'sqrt']

# maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# minimum number of samples required to split a node
min_sample_split = [2, 5, 10]

# minimum number of samples required at each leaf node
min_sample_leaf = [1, 2, 4]

# method of selecting samples for training each tree
bootstrap = [True, False]

# create the random grid
random_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_sample_split,
              'min_samples_leaf': min_sample_leaf,
              'bootstrap': bootstrap}

#pprint(random_grid)

In [ ]:
# use the random grid to search for best hyperparameters
rf = RandomForestClassifier()

# random search of parameters using 3 fold corss validation 
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose = 2, random_state = 15, n_jobs = -1)

# fit the random search model
rf_random.fit(X, y)

rf_random.best_params_

In [ ]:
rf = RandomForestClassifier(n_estimators = 72, random_state = 15, min_samples_split=10, min_samples_leaf=4, max_features='sqrt', 
                  max_depth=30, bootstrap= False)
rf.fit(X_train, y_train)
#rf.score(X_test, y_test)
print('the random forest score without CV: ', rf.score(X_test, y_test))

In [10]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose = 2, random_state = 15, n_jobs = -1)
rf_random.fit(X, y)

print('the random forest score with CV: ', rf_random.score(X, y))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
the random forest score with CV:  0.9230769230769231


## nested grid search with CV

In [12]:
from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}# Create a based model
rf = RandomForestClassifier()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [15]:
# Fit the grid search to the data
grid_search.fit(X,y)
grid_search.best_params_

Fitting 3 folds for each of 288 candidates, totalling 864 fits


{'bootstrap': True,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 5,
 'min_samples_split': 8,
 'n_estimators': 100}

In [16]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, test_features, test_labels)

NameError: name 'evaluate' is not defined

## evaluation

In [ ]:
# Confusion matrix
confusion = metrics.confusion_matrix(y_test, y_pred)
print(confusion)

In [ ]:
import seaborn as sns

heatmap = sns.heatmap(confusion, annot=True, cmap='Oranges')

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(),
                            rotation=0, ha='right')

heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(),
                            rotation=0, ha='right')

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('RF Model Results')
plt.show()

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt

y_pred_proba = model.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
TP = confusion[1,1] # true positive
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model

print('the sensitivity if this model is: ', TP / float(TP+FN))

In [ ]:
# Let us calculate specificity

print('the specificity if this model is: ', TN / float(TN+FP))

In [ ]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
y_score1 = model.predict_proba(X_test)[:,1]
y_score2 = model.predict_proba(X_test)[:,1]
false_positive_rate1, true_positive_rate1, threshold1 = roc_curve(y_test, y_score1)
false_positive_rate2, true_positive_rate2, threshold2 = roc_curve(y_test, y_score2)
print('roc_auc_score for DecisionTree: ', roc_auc_score(y_test, y_score1))

### repeating RF with k best important features (ExtraTreesClassifier method)
#### 50-best: importance>0.005

In [ ]:
model = ExtraTreesClassifier()
model.fit(X,y)
#print(model.feature_importances_)

In [ ]:
feat_importances = pd.Series(model.feature_importances_ , index = df.columns[2:])
feat_importances.nlargest(50).plot(kind = 'barh')
plt.show()

In [ ]:
feat_imp = pd.DataFrame(feat_importances)
feat_imp.reset_index(inplace = True)
feat_imp.rename(columns={'index': 'region', 0:'importance'}, inplace = True)
feat_imp.sort_values(by='importance', ascending=False,inplace=True)
feat_imp

In [ ]:
feat_imp = feat_imp.iloc[0:50,:]
feat_imp=feat_imp.reset_index()
feat_imp.drop('index', axis=1, inplace=True)
feat_imp.head()

In [ ]:
feat_imp.to_csv('top50feat.csv')

In [ ]:
df_with25imp = df[[col for col in feat_imp['region']]]
df_with25imp.insert(0, 'Subject_ID', df['Subject_ID'])
df_with25imp.insert(1, 'diagadhd', df['diagadhd'])
df_with25imp.head()

##### now applying the Random Forest:

In [ ]:
# train-test split evaluation of xgboost model
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# split data into X and y
#X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,2:], data['diagadhd'], test_size=0.3)
training_set, test_set = train_test_split(df_with25imp, test_size = 0.2, random_state = 3)
                                         
X_train = training_set.iloc[:,2:].values
y_train = training_set.iloc[:,1].values
X_test = test_set.iloc[:,2:].values
y_test = test_set.iloc[:,1].values

# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
### from sklearn.model_selection import RandomizedSearchCV

# number of trees in RF
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 91)]

# number of features to consider at every split
max_features = ['auto', 'sqrt']

# maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# minimum number of samples required to split a node
min_sample_split = [2, 5, 10]

# minimum number of samples required at each leaf node
min_sample_leaf = [1, 2, 4]

# method of selecting samples for training each tree
bootstrap = [True, False]

# create the random grid
random_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_sample_split,
              'min_samples_leaf': min_sample_leaf,
              'bootstrap': bootstrap}

#pprint(random_grid)

In [ ]:
# use the random grid to search for best hyperparameters
rf = RandomForestClassifier()

# random search of parameters using 3 fold corss validation 
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose = 2, random_state = 15, n_jobs = -1)

# fit the random search model
rf_random.fit(X, y)

rf_random.best_params_

In [ ]:
rf = RandomForestClassifier(n_estimators = 68, random_state = 25, min_samples_split=2, min_samples_leaf=2, max_features='auto', max_depth = 100, bootstrap= False)
rf.fit(X_train, y_train)
print('the random forest score without CV: ', rf.score(X_test, y_test))

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose = 2, random_state = 35, n_jobs = -1)
rf_random.fit(X_train, y_train)

print('the random forest score with CV: ', rf_random.score(X_test, y_test))

## Evaluation

In [ ]:
# Confusion matrix
confusion = metrics.confusion_matrix(y_test, y_pred)
print(confusion)

In [ ]:
import seaborn as sns
heatmap = sns.heatmap(confusion, annot=True, cmap='Oranges')

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(),
                            rotation=0, ha='right')

heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(),
                            rotation=0, ha='right')

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('RF Model Results')
plt.show()

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt

y_pred_proba = model.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
y_score1 = model.predict_proba(X_test)[:,1]
y_score2 = model.predict_proba(X_test)[:,1]
false_positive_rate1, true_positive_rate1, threshold1 = roc_curve(y_test, y_score1)
false_positive_rate2, true_positive_rate2, threshold2 = roc_curve(y_test, y_score2)
print('roc_auc_score for DecisionTree: ', roc_auc_score(y_test, y_score1))

In [ ]:
TP = confusion[1,1] # true positive
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model

TP / float(TP+FN)

In [ ]:
# Let us calculate specificity

TN / float(TN+FP)